In [1]:
# Importação de bibliotecas necessárias para análise de dados e modelagem
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer


import os
 # Listar todos os arquivos no diretório de entrada
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
# Leitura dos dados de treinamento e teste
df_train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

# Atribui a coluna 'Transported' como False para o dataset de teste (não existe no teste)
df_test['Transported'] = False
# Combina os datasets de treino e teste para facilitar o processamento
df = pd.concat([df_train, df_test], sort = False)
# Remove colunas 'Name' e 'PassengerId' que não são necessárias para a modelagem
df.drop(['Name', 'PassengerId'], axis = 1, inplace = True)
# Verificação do formato dos dados após a junção dos datasets
df.head()


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [3]:
# Confirmação de que o número de linhas no dataframe combinado é igual à soma das linhas dos dados de treino e teste
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [4]:
# Verificação de valores ausentes (NaN) nas colunas do dataframe
df.isna().sum()

HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
dtype: int64

In [5]:
# Extração de informações da coluna 'Cabin' para gerar as colunas 'Deck', 'Num' e 'Side'
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand = True)
# Remove a coluna original 'Cabin'
df = df.drop(columns = ['Cabin']) # Remove a coluna original 'Cabin'
# Preenchimento dos valores ausentes nas novas colunas
df['Deck'] = df['Deck'].fillna('U')
df['Num'] = df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')

In [6]:
# Contagem das ocorrências de cada valor na coluna 'Destination'
df['Destination'].value_counts()

Destination
TRAPPIST-1e      8871
55 Cancri e      2641
PSO J318.5-22    1184
Name: count, dtype: int64

In [7]:
# Mapeamento dos valores da coluna 'Deck' para números inteiros
df['Deck'] = df['Deck'].map({'G' : 0, 'F' : 1, 'E' : 2, 'D' : 3, 'C' : 4, 'B' : 5, 'A' : 6, 'U' : 7, 'T' : 8})
# Mapeamento dos valores da coluna 'Side' para números inteiros
df['Side'] = df['Side'].map({'U' : -1, 'P' : 1, 'S' : 2})

In [8]:
# Lista de colunas que precisam de preenchimento de valores ausentes
impute_lis = ['Age', 'VIP', 'Num', 'CryoSleep', 'Side', 'Deck', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
# Colunas que não precisam de imputação de valores ausentes
rest = list(set(df.columns) - set(impute_lis))
df_rest = df[rest]
# Preenchimento dos valores ausentes nas colunas selecionadas usando o KNNImputer
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_lis])
df_imputed = pd.DataFrame(df_imputed, columns = impute_lis)
# Combina os dados imputados com as colunas que não foram alteradas
df = pd.concat([df_rest.reset_index(drop = True), df_imputed.reset_index(drop = True)], axis = 1)

In [9]:
# Preenchimento de valores ausentes nas colunas 'HomePlanet' e 'Destination'
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
# Criação de variáveis dummies para as colunas categóricas 'HomePlanet' e 'Destination'
category_colls = ['HomePlanet', 'Destination']
for col in category_colls:
    df = pd.concat([df, pd.get_dummies(df[col], prefix = col)], axis = 1)

In [10]:
# Remoção das colunas originais 'HomePlanet' e 'Destination' após a criação das dummies
df = df.drop(columns = category_colls)

In [11]:
df.head()

,Transported,Age,VIP,Num,CryoSleep,Side,Deck,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_U,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_U
0,False,39.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,False,False,True,False
1,True,24.0,0.0,0.0,0.0,2.0,1.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,True,False
2,False,58.0,1.0,0.0,0.0,2.0,6.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,True,False
3,False,33.0,0.0,0.0,0.0,2.0,6.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,True,False
4,True,16.0,0.0,1.0,0.0,2.0,1.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,True,False


In [12]:
# Engenharia de características: criação de novas variáveis baseadas nas colunas relacionadas a gastos
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['amt_spent'] = df[bill_cols].sum(axis = 1) # Soma dos gastos
df['std_amt_spent'] = df[bill_cols].std(axis = 1) # Desvio padrão dos gastos
df['mean_amt_spent'] = df[bill_cols].mean(axis = 1) # Média dos gastos

# Criação de novas colunas baseadas em combinações específicas de variáveis
df['3_high_cols'] = df['CryoSleep'] + df['HomePlanet_Europa'] + df['Destination_55 Cancri e']
df['3_low_cols'] = df['mean_amt_spent'] + df['amt_spent'] + df['HomePlanet_Earth']

In [13]:
# Correlação das variáveis com a coluna 'Transported' para verificar quais são mais influentes
df.corr()['Transported'].sort_values(ascending = False)

Transported                  1.000000
CryoSleep                    0.324373
3_high_cols                  0.284177
HomePlanet_Europa            0.131977
Destination_55 Cancri e      0.083625
Deck                         0.077959
Side                         0.059872
FoodCourt                    0.034746
HomePlanet_U                 0.006403
HomePlanet_Mars              0.005643
ShoppingMall                 0.004154
Destination_PSO J318.5-22    0.000760
Destination_U               -0.000554
VIP                         -0.018720
Num                         -0.035240
Age                         -0.050520
Destination_TRAPPIST-1e     -0.072731
HomePlanet_Earth            -0.119644
std_amt_spent               -0.121173
mean_amt_spent              -0.140452
amt_spent                   -0.140452
3_low_cols                  -0.140476
VRDeck                      -0.142783
Spa                         -0.154832
RoomService                 -0.174781
Name: Transported, dtype: float64

In [14]:
# Separação do dataframe de volta em treino e teste
df_train, df_test = df[:df_train.shape[0]], df[df_train.shape[0]:]
df_test = df_test.drop(columns = 'Transported')
df_train.shape, df_test.shape

((8693, 25), (4277, 24))

In [15]:
# Importação de modelos de aprendizado de máquina para classificação
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
# Separação dos dados de treinamento em variáveis independentes (X) e dependentes (y)
X = df_train.drop(columns = 'Transported')
y = df_train['Transported']

# Divisão dos dados de treinamento em conjunto de treino e conjunto de teste (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Criação de instâncias dos modelos a serem testados
model_1 = LogisticRegression()
model_2 = DecisionTreeClassifier()
model_3 = RandomForestClassifier()
model_4 = XGBClassifier()
model_5 = LGBMClassifier()

In [17]:
# Treinamento e avaliação de cada modelo
model_1.fit(X_train, y_train)
pred = model_1.predict(X_test)
accuracy_score(y_test, pred) # Avaliação do modelo 1

# Esse warning ocorre porque o solver de otimização (lbfgs) não conseguiu convergir dentro do número máximo de iterações padrão. Como não influencia diretamente no projeto, pode ser ignorado por enquanto.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7705577918343876

In [18]:
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
accuracy_score(y_test, pred) # Avaliação do modelo 2

0.7452558941920644

In [19]:
model_3.fit(X_train, y_train)
pred = model_3.predict(X_test)
accuracy_score(y_test, pred) # Avaliação do modelo 3

0.7878090856814262

In [20]:
model_4.fit(X_train, y_train)  
pred = model_4.predict(X_test)
accuracy_score(y_test, pred)  # Avaliação do modelo 4

0.7941345600920069

In [21]:
model_5.fit(X_train, y_train)
pred = model_5.predict(X_test)
accuracy_score(y_test, pred)   # Avaliação do modelo 5

# Esse warning abaixo é mostrado porque o LightGBM encontrou espaços nos nomes das colunas e os substituiu por underlines. Além disso há algumas informações.

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2703
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


0.79700977573318

In [22]:
import pickle

# Salvar o modelo treinado usando pickle
model_save_path = "spaceship_titanic.pkl"  # Defina o caminho onde deseja salvar o modelo
with open(model_save_path, 'wb') as model_file: # Usando 'pickle' para salvar o modelo em um arquivo binário
    pickle.dump(model_5, model_file) # Salva o modelo 'model_5' no arquivo

print(f"Modelo salvo em: {model_save_path}")


Modelo salvo em: spaceship_titanic.pkl


In [23]:
# Leitura do arquivo de dados de teste
df_dummy = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv") # Dados de teste, sem a coluna 'Transported'
# Previsão com o modelo treinado
pred = model_5.predict(df_test) # Faz as previsões para o conjunto de teste

# Criação de um DataFrame para armazenar os resultados
final = pd.DataFrame()
final['PassengerId'] = df_dummy['PassengerId'] # Coloca os IDs dos passageiros no DataFrame
final['Transported'] = pred # Coloca as previsões de 'Transported' no DataFrame

# Salvar as previsões em um arquivo CSV para submissão ao Kaggle
final.to_csv('submission.csv', index = False)